In [2]:
# import gdown
import zipfile
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import xml.etree.ElementTree as ET
import torch
from torchvision.transforms import v2
import albumentations as A
from torch.utils.data import DataLoader, Dataset
from albumentations.pytorch import ToTensorV2
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
# from torchsummary import summary

In [3]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# Initialize webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(frame_rgb)
    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x1, y1 = int(bboxC.xmin * iw), int(bboxC.ymin * ih)
            x2, y2 = int((bboxC.xmin + bboxC.width) * iw), int((bboxC.ymin + bboxC.height) * ih)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imshow('Face Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


h:\Project\Filter\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [4]:
class Filter():
    def __init__(self, model, transform):
        self.landmark_detect = model
        self.transform = transform
        self.face_detect = dlib.get_frontal_face_detector()
        self.landmark_detect.eval()
        
    def landmarks_detect(self, image):
        height, width, channels = image.shape
        
        image = self.transform(image=image)['image']
        in_height, in_width, in_channel = image.shape
        image = image.unsqueeze(0)
        landmarks = self.landmark_detect(image)
        landmarks = landmarks.detach().numpy()
        landmarks = landmarks.reshape(68, 2)
        
        # convert to 0-224 axis
        landmarks = (landmarks + 0.5) * in_width
        # convert to original axis
        landmarks[:, 0] = landmarks[:, 0] / in_width * width
        landmarks[:, 1] = landmarks[:, 1] / in_width * height
        
        return landmarks
        
        
    def full_landmarks_detect(self, image_path):
        # Load the image (np.ndarray)
        image = cv2.imread(image_path)

        # Convert BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Detect faces in the image
        faces = self.face_detect(image_rgb, 1)

        # Draw rectangles around the detected faces
        for i, face in enumerate(faces):
            left, top, right, bottom = (face.left(), face.top(), face.right(), face.bottom())
            landmarks = self.landmarks_detect(image_rgb[top:bottom, left:right])
            landmarks = landmarks + [left, top]
            
            cv2.rectangle(image_rgb, (left, top), (right, bottom), (0, 255, 0), 2)
            for x, y in landmarks:
                cv2.circle(image_rgb, (int(round(x)), int(round(y))), radius=1, color=(0, 255, 0), thickness=-1)

        # Display the image with detected faces
        plt.imshow(image_rgb)
        plt.show()

In [5]:
transform_pred = A.Compose([
    A.Resize(height=224, width=224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

AttributeError: module 'dlib' has no attribute 'get_frontal_face_detector'